Code implementation based on the [video tutorial](https://www.youtube.com/watch?v=kCc8FmEb1nY&list=WL&index=6) provided by Andrej Karpathy.

In [36]:
# Imports
import os
import wget
import torch
import torch.nn as nn
from torch.nn import functional as F

In [37]:
# Download the Shakespeare dataset and save as input.txt if file doesn't exist.
if 'input.txt' not in os.listdir():
    url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    wget.download(url, 'input.txt')
    print('Downloaded input.txt')

In [38]:
# read it in to inspect it
with open('input.txt', 'r') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [39]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.


In [40]:
# Here are all the unique characters that occur in this text.
# This approach creates a vocabulary where each character is a token. ChatGPT and other
# language models usually use a sub-word vocabulary, which trades-off between large input 
# context window and large vocabulary vectors
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [41]:
# Strategy to tokenize the text into a vector of integers, where each integer represents a character
# from the vocabulary. The encode method converts the character into its integer representation, while
# the decode method converts the integer back to the character. Both methods accept vectors of characters
# or integers, respectively. stoi and itos are lookup tables used for conversion.
stoi = {ch: i for i, ch in enumerate(chars)}  # String to Integer (stoi)
itos = {i: ch for i, ch in enumerate(chars)}  # Integer to String (itos)
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [42]:
# Now we encode the entire text dataset and store it into a torch.Tensor variable.
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# This is the data format GPT will look at, the number representation for each character
# notice how in the output 1s probably represent spaces.
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [43]:
# Splits the data into a training and validation sets
n = int(0.9 * len(data))  # 90% of the data for training, rest for validation
train_data = data[:n]
val_data = data[n:]

In [44]:
# For training the model we use sampled chunks of the training set (data variable) and never the entire dataset, 
# because it would be computationally expensive to use the entire set. To define the maximum chunk size the
# variable block_size size defines the maximum length of characters a block can have. This value can be adjusted,
# but Andrej usually uses 8.
block_size = 8
# Notice the blocks are inclusive regarding the last character. In a block for training each additional character
# represents another training sample, because we reuse the same vector by incrementing the prediction of the next
# character. So in a training block of size 9, we have 8 training samples, each one trying to predict the next char.
print("Example of trainable block:", train_data[:block_size + 1])

# Example of training samples within a single block of size 8. Blocks are not only defined for computational reasons, but also
# to make sure the model learns to predict since a small context (1 char) up to a larger context (8 chars). So the transformer
# learns to sample during generation from the first character.
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]  # P(target | context)
    print(f"when input is {context} the target is {target}")

Example of trainable block: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [45]:
# Now we define batches of blocks for training. This is purely for efficiency reasons because training models on GPUs take the 
# advantage of parallelism. GPUs are very good at parallelize operations, specially those that are not dependent on each other.
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum content length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # gets batch_size random offsets of size block_size
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print("inputs:", xb.shape, xb)
print("targets:", yb.shape, yb)
print('----------------')

for b in range(batch_size):  # batch dimension
    for t in range(block_size):  # time dimension
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"batch {b} when input is {context} the target is {target}")

inputs: torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------
batch 0 when input is tensor([24]) the target is 43
batch 0 when input is tensor([24, 43]) the target is 58
batch 0 when input is tensor([24, 43, 58]) the target is 5
batch 0 when input is tensor([24, 43, 58,  5]) the target is 57
batch 0 when input is tensor([24, 43, 58,  5, 57]) the target is 1
batch 0 when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
batch 0 when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
batch 0 when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
batch 1 when input is tensor([44]) the target is 53
batch 1 when

In [46]:
print("Input to the transformer model:")
print(xb)

Input to the transformer model:
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [47]:
torch.manual_seed(1337)


# This is not GPT yet, this is one of the simples possible language models
class BigramLanguageModel(nn.Module):
    # This model is ridiculous in the sense it receives the whole sequence of tokens for prediction but
    # only uses the last character to predict the next one. This id different from GPT, which uses all the
    # tokens to predict the next one.

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx represent a token from the input sequence (xb). For example, if idx is the letter 43 (from the previous example)
        # then idx will pluck out the vector corresponding to that row idx from the embeddings matrix as the embeddings
        # to represent the token 43.

        # B = batch_size, T = time/block_size, C = channel/vocab_size
        # The idx and targets are both (B, T) tensor of integers.

        # This logits are basically predicting what comes next based on a single token
        # In this bigram language model the tokens are not talking to each other (considering the sequence),
        # they are just been looked individually to predict the next token.

        # Notice how the computation is carried out in parallel for all the batch_size elements
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            # Quality of the logits with respect to the targets. How well are we predicting the next token compared to the
            # target tokens (notice the numbers are not between 0 and 1, they are based on the vocabulary integer representations).
            # However, to make it work using the functional API we need to reshape the logits as the expected shape by tensor flow.
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # Stretch the sequence tokens and batches into a single dimension
            targets = targets.view(B * T)  # or -1
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # B = batch_size, T = time/block_size, C = channel/vocab_size
        # idx is (B,T) array of indices in the current context (same input from forward)
        # The idea is to generate tokens (..., T+1, T+2, ..., T+max_new_tokens) for each 
        # batch in B in add to the current context.
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)  # loss is ignored for generation
            # focus only on the last time step, notice the logits become three-dimensional during the loop
            logits = logits[:, -1, :]  # Becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, "(batch size, block size, vocab size)")
print("Loss:", loss)

# Generation
idx = torch.zeros((1, 1), dtype=torch.long)  # Prompt (1 batch, 1 token)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))  # Print all the sequence from the single batch dimension

torch.Size([32, 65]) (batch size, block size, vocab size)
Loss: tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [48]:
# Create optimizer to train the model, learning rate is usually set to 1e-3, but for smaller networks it could be set a little higher.
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10_000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 1000 == 0:
        print(loss.item())

print("Final loss:", loss.item())

4.692410945892334
3.7683708667755127
3.2364423274993896
2.892672061920166
2.70308780670166
2.513338327407837
2.486689329147339
2.5133867263793945
2.4423880577087402
2.3939623832702637
Final loss: 2.3796486854553223


In [49]:
# Let's generate again after the model has been trained
# This is still not going to be shakespeare, but it should be better than before
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


llo br. ave aviasurf my, mayo t ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;

Whe, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty dedo bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
By bre ndy; by s afreanoo adicererupa anse tecor


# The mathematical trick in self attention

In [68]:
# VERSION 1
# consider the toy example to facilitate understanding of the self attention mechanism used by transformers.

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
print("x = ", x.shape, "\n")

# Now for this simple example, we want to represent the attention from token T to the previous 
# tokens T-1 down to 0 by taking the average. This is a simple example of self-attention. This is
# faulty because it loses important information about spacial position of the previous tokens in
# the sentence. Notice we only calculate the attention for the previous tokens, not the future ones.
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))  #bow = bag of words, a common term when you are averaging over words
for b in range(B):  # for each batch
    for t in range(T):  # for each token
        xprev = x[b, :t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

print(f"bag of words for b_0 and t_0:\n{xbow[0, 0]}\nfrom sequence\n{x[0, :1]}\n")
print(f"bag of words for b_0 and t_1:\n{xbow[0, 1]}\nfrom sequence\n{x[0, :2]}\n")
print(f"bag of words for b_0 and t_2:\n{xbow[0, 2]}\nfrom sequence\n{x[0, :3]}\n")
print("...")

x =  torch.Size([4, 8, 2]) 

bag of words for b_0 and t_0:
tensor([ 0.1808, -0.0700])
from sequence
tensor([[ 0.1808, -0.0700]])

bag of words for b_0 and t_1:
tensor([-0.0894, -0.4926])
from sequence
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152]])

bag of words for b_0 and t_2:
tensor([ 0.1490, -0.3199])
from sequence
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255]])

...


In [72]:
# The previous form of calculating the bag of words is not efficient, because it is not parallelized.
# The following example shows a more efficient way of achieving the same.
# This is the mathematical trick used by the self-attention mechanism in transformers.
torch.manual_seed(42)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print("Matrix multiplication:")
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a = torch.tril(a)  # this returns the lower triangular part of the matrix
c = a @ b
print("Matrix multiplication with lower triangular:")
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

# Now, by dividing a by the sum of each row, we can get the average of the previous tokens
a = torch.tril(a)
a = a / a.sum(-1, keepdim=True)
c = a @ b
print("Average of the previous tokens:")
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

Matrix multiplication:
a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
Matrix multiplication with lower triangular:
a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
Average of the previous tokens:
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [78]:
# VERSION 2
# Now using the trick from the previous block for the toy example from the before last block
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(wei)
xbow2 = wei @ x  # (T, T) @ (B, T, C) = (B, T, C) torch will resolve wei to (1, T, T) and do broadcasting to (B, T, T)
torch.allclose(xbow, xbow2)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [80]:
# VERSION 3: use softmax
# This method is more interesting because allows wei to define how much attention we want to give to each token in the 
# real self-attention mechanism. In other words, when we cross all tokens against all other tokens, the wei will define
# how much affinity (or attention) that token should have to the other token.
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))  # makes all elements where tril == 0 to -inf
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [87]:
# VERSION 4: self-attention using Query, Key, Value
# There's one query and key for each token, the idea is query ask what is this token looking for in the other tokens, while they
# keys represent what how much other token can answer that question for the first token. Then the affinity between the question and
# token is used to weight the value of the token.
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, head_size)
q = query(x)  # (B, T, head_size)
# wei = affinities, to transpose the last two dimensions (not the batch dimension)
wei = q @ k.transpose(-2, -1)  # (B, T, head_size) @ (B, head_size, T) = (B, T, T)
# so that the variance is between 0-1 and not between 0-head_size, this helps softmax to avoid
# converging to a hot vector if the values are too positive or too negative.
wei = wei * head_size ** -0.5

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)  # (T, T) how much attention each token should give to the other tokens from the past

# x can be seen as private information, and if it happens this token is interesting to another token the value is 
# actually what is communicated
v = value(x)
out = wei @ v

print(out.shape)
print(wei[0])

torch.Size([4, 8, 16])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)


Notes:
* Attention is a *communication mechanism*. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
* Each example across batch dimension is of course processed completely independently and never "talk" to each other.
* In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate.
* Self-attention means the Q, K, and V come from the same source X. But in an encoder-decoder transformer, the "cross-attention" in the decoder block means the Q, and V come from X in the decoder block, but K comes from another source, an encoder block.